In [41]:
import os
import cv2
from PIL import Image
import clip
import torch
import math
import numpy as np
import plotly.express as px
import datetime
from IPython.core.display import HTML
import tensorflow as tf

In [42]:
def search_video(search_query, display_heatmap=True, display_results_count=10):

  # Encode and normalize the search query using CLIP
  with torch.no_grad():
    text_features = model.encode_text(clip.tokenize(search_query).to(device))
    text_features /= text_features.norm(dim=-1, keepdim=True)

  # Compute the similarity between the search query and each frame using the Cosine similarity
  similarities = (100.0 * video_features @ text_features.T)
  values, best_photo_idx = similarities.topk(display_results_count, dim=0)
#   print(values)
  sum_values = torch.sum(values, axis = 0)
  ave_values = sum_values / display_results_count
  print(ave_values)
  test_results.append(zip(i.split('/')[-1].replace('.mp4', ''), ave_values))
#   print(sum_values)
#   print(type(values))

  # Display the heatmap
#   if display_heatmap:
#     print("Search query heatmap over the frames of the video:")
#     fig = px.imshow(similarities.T.cpu().numpy(), height=50, aspect='auto', color_continuous_scale='viridis')
#     fig.update_layout(coloraxis_showscale=False)
#     fig.update_xaxes(showticklabels=False)
#     fig.update_yaxes(showticklabels=False)
#     fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
#     fig.show()
#     print()

  # Display the top 3 frames
#   b = 0
#   for frame_id in best_photo_idx:
#     image = video_frames[frame_id]
#     image.save(os.path.join(root,str(frame_id) + ".jpg"))
#     display(video_frames[frame_id])
    
    

    # Find the timestamp in the video and display it
#     seconds = round(frame_id.cpu().numpy()[0] * N / fps)
#     display(HTML(f"Found at {str(datetime.timedelta(seconds=seconds))} (<a target=\"_blank\" href=\"{video_url}&t={seconds}\">link</a>)"))

In [43]:
root = '/home/qls/Downloads/james_huang_project/video'
sub_root = []
video_1_files = []
video_2_files = []
video_3_files = []
test_results = []
N = 120
file1 = os.listdir(root)
for i in file1:
    sub_root.append(os.path.join(root,i))
for i in sub_root:
    if i.split('/')[-1] == '1':
        for j in os.listdir(i):
            video_1_files.append(os.path.join(i,j))
    if i.split('/')[-1] == '2':
        for j in os.listdir(i):
            video_2_files.append(os.path.join(i,j))
    if i.split('/')[-1] == '3':
        for j in os.listdir(i):
            video_3_files.append(os.path.join(i,j))
# print(video_1_files)
# print(video_2_files)
# print(video_3_files)

In [44]:
# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

for i in video_1_files:
    # The frame images will be stored in video_frames
    video_frames = []

    # Open the video file
    capture = cv2.VideoCapture(i)
    fps = capture.get(cv2.CAP_PROP_FPS)

    current_frame = 0
    while capture.isOpened():
      # Read the current frame
      ret, frame = capture.read()

      # Convert it to a PIL image (required for CLIP) and store it
      if ret == True:
        video_frames.append(Image.fromarray(frame[:, :, ::-1]))
      else:
        break

      # Skip N frames
      current_frame += N
      capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

    # Print some statistics
#     print(f"Frames extracted: {len(video_frames)}")
    
    # You can try tuning the batch size for very large videos, but it should usually be OK
    batch_size = 256
    batches = math.ceil(len(video_frames) / batch_size)

    # The encoded features will bs stored in video_features
    video_features = torch.empty([0, 512], dtype=torch.float16).to(device)

    # Process each batch
    for j in range(batches):
      print(f"Processing batch {j+1}/{batches}")

      # Get the relevant frames
      batch_frames = video_frames[j*batch_size : (j+1)*batch_size]

      # Preprocess the images for the batch
      batch_preprocessed = torch.stack([preprocess(frame) for frame in batch_frames]).to(device)

      # Encode with CLIP and normalize
      with torch.no_grad():
        batch_features = model.encode_image(batch_preprocessed)
        batch_features /= batch_features.norm(dim=-1, keepdim=True)

      # Append the batch to the list containing all features
      video_features = torch.cat((video_features, batch_features))

    # Print some stats
#     print(f"Features: {video_features.shape}")
    
    print(i.split('/')[-1].replace('.mp4', ''))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    search_video("cadillac car")
print(test_results)

    


Processing batch 1/1
1.3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([26.1250], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
1.1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([31.7031], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
1.4
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([27.1250], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
1.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([28.2031], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
1.9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([32.3125], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
1.10
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([27.8281], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
1.7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([23.5000], device='

In [45]:
for i in video_2_files:
    # The frame images will be stored in video_frames
    video_frames = []

    # Open the video file
    capture = cv2.VideoCapture(i)
    fps = capture.get(cv2.CAP_PROP_FPS)

    current_frame = 0
    while capture.isOpened():
      # Read the current frame
      ret, frame = capture.read()

      # Convert it to a PIL image (required for CLIP) and store it
      if ret == True:
        video_frames.append(Image.fromarray(frame[:, :, ::-1]))
      else:
        break

      # Skip N frames
      current_frame += N
      capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

    # Print some statistics
#     print(f"Frames extracted: {len(video_frames)}")
    
    # You can try tuning the batch size for very large videos, but it should usually be OK
    batch_size = 256
    batches = math.ceil(len(video_frames) / batch_size)

    # The encoded features will bs stored in video_features
    video_features = torch.empty([0, 512], dtype=torch.float16).to(device)

    # Process each batch
    for j in range(batches):
      print(f"Processing batch {j+1}/{batches}")

      # Get the relevant frames
      batch_frames = video_frames[j*batch_size : (j+1)*batch_size]

      # Preprocess the images for the batch
      batch_preprocessed = torch.stack([preprocess(frame) for frame in batch_frames]).to(device)

      # Encode with CLIP and normalize
      with torch.no_grad():
        batch_features = model.encode_image(batch_preprocessed)
        batch_features /= batch_features.norm(dim=-1, keepdim=True)

      # Append the batch to the list containing all features
      video_features = torch.cat((video_features, batch_features))

    # Print some stats
#     print(f"Features: {video_features.shape}")
    
    print(i.split('/')[-1].replace('.mp4', ''))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    search_video("anaconda")
print(test_results)

Processing batch 1/1
2.6
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([27.2031], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
2.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([28.8750], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
2.7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([20.2188], device='cuda:0', dtype=torch.float16)
Processing batch 1/6
Processing batch 2/6
Processing batch 3/6
Processing batch 4/6
Processing batch 5/6
Processing batch 6/6
2.10
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([28.4688], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
2.9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([29.1719], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
2.3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([25.5625], device='cuda:0', dtype=torch.float16)
Processing b

In [46]:
for i in video_3_files:
    # The frame images will be stored in video_frames
    video_frames = []

    # Open the video file
    capture = cv2.VideoCapture(i)
    fps = capture.get(cv2.CAP_PROP_FPS)

    current_frame = 0
    while capture.isOpened():
      # Read the current frame
      ret, frame = capture.read()

      # Convert it to a PIL image (required for CLIP) and store it
      if ret == True:
        video_frames.append(Image.fromarray(frame[:, :, ::-1]))
      else:
        break

      # Skip N frames
      current_frame += N
      capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

    # Print some statistics
#     print(f"Frames extracted: {len(video_frames)}")
    
    # You can try tuning the batch size for very large videos, but it should usually be OK
    batch_size = 256
    batches = math.ceil(len(video_frames) / batch_size)

    # The encoded features will bs stored in video_features
    video_features = torch.empty([0, 512], dtype=torch.float16).to(device)

    # Process each batch
    for j in range(batches):
      print(f"Processing batch {j+1}/{batches}")

      # Get the relevant frames
      batch_frames = video_frames[j*batch_size : (j+1)*batch_size]

      # Preprocess the images for the batch
      batch_preprocessed = torch.stack([preprocess(frame) for frame in batch_frames]).to(device)

      # Encode with CLIP and normalize
      with torch.no_grad():
        batch_features = model.encode_image(batch_preprocessed)
        batch_features /= batch_features.norm(dim=-1, keepdim=True)

      # Append the batch to the list containing all features
      video_features = torch.cat((video_features, batch_features))

    # Print some stats
#     print(f"Features: {video_features.shape}")
    
    print(i.split('/')[-1].replace('.mp4', ''))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    search_video("cadillac car")
print(test_results)

Processing batch 1/1
3.2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([18.8438], device='cuda:0', dtype=torch.float16)
Processing batch 1/2
Processing batch 2/2
3.10
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([16.5312], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
3.4
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([21.2812], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
3.8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([18.9062], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
3.9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([20.4062], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
3.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tensor([18.9062], device='cuda:0', dtype=torch.float16)
Processing batch 1/1
3.6
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
tenso